In [5]:
import random
import itertools
import numpy as np
import json
import pprint
from __future__ import print_function


file = np.load('merged_train.npy')
x = file[0:1000]
with open('merged_train.json') as data_file:    
    data = json.load(data_file)
y = data[0:1000]

n = int(0.9*len(x))
train_x = x[:n]
test_x = x[n:]

train_y = y[:n]
test_y = y[n:]

f = open('sample_merged_train.json','w')
for item in y:
    f.write("%s\n" % item)




# x = []
# y = []
# for i in xrange(1, 50):
#     x.append([0] * i)
#     y.append(0)
#     k = random.randint(0, len(x[-1]) - 1)
#     x.append(list(x[-1]))
#     x[-1][k] = 1
#     y.append(1)

# zipped = zip(x, y)
# random.shuffle(zipped)
# x, y = zip(*zipped)
# n = int(0.9 * len(zipped))

# train_x, train_y = x[:n], y[:n]
# test_x, test_y = x[n:], y[n:]

In [30]:
a = {}
for captions in data:
    for sen in captions:
        for word in sen[1]:
            if not word in a:
                a[word] = len(a)

def num_to_vec(num):
    vec = np.zeros(len(a))
    vec[num] = 1
    return vec
    
def vec_to_num(vec):
    return np.where(vec == 1)
    
vec = num_to_vec(2)
print vec
print vec_to_num(vec) 
                

[ 0.  0.  1. ...,  0.  0.  0.]
(array([2], dtype=int64),)


In [2]:
import theano
from theano import tensor as T

class EmbeddingLayer(object):
    def __init__(self, embedding_init):
        self.embedding_matrix = theano.shared(embedding_init())

    def get_output_expr(self, input_expr):
        return self.embedding_matrix[input_expr]

    def get_parameters(self):
        return [self.embedding_matrix]


class RnnLayer(object):
    def __init__(self, w_init, u_init):
        self.W = theano.shared(w_init())
        self.U = theano.shared(u_init())

    def get_output_expr(self, input_sequence):
        h0 = T.zeros((self.W.shape[0], ))

        h, _ = theano.scan(fn=self.__get_rnn_step_expr,
                           sequences=input_sequence,
                           outputs_info=[h0])
        return h

    def __get_rnn_step_expr(self, x_t, h_tm1):
        return T.tanh(T.dot(h_tm1, self.W) + T.dot(x_t, self.U))

    def get_parameters(self):
        return [self.W, self.U]

    
class LogisticRegresion(object):
    def __init__(self, w_init):
        self.W = theano.shared(w_init())
        
    def get_output_expr(self, input_expr):
        pre_softmax_expr = T.dot(input_expr, self.W)
        return 1 / (1 + T.exp(pre_softmax_expr))

    def get_parameters(self):
        return [self.W]
    

def get_sgd_updates(cost, params, lr=0.01):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        updates.append([p, p - lr * g])
    return updates    

[u'data/datasets/mscoco/train2014/COCO_train2014_000000191666.jpg', [[u'A', u'white', u'sink', u'sitting', u'next', u'to', u'a', u'toilet', u'in', u'a', u'bathroom', u'.'], [u'A', u'bathroom', u'has', u'a', u'sink', u',', u'toilet', u'and', u'an', u'orange', u'bucket', u'in', u'it', u'.'], [u'A', u'very', u'dingy', u'looking', u'tiled', u'bathroom', u'with', u'a', u'sink', u'and', u'toilet', u'.'], [u'A', u'small', u'bathroom', u'with', u'a', u'commode', u'and', u'sink', u',', u'and', u'empty', u'corner', u'.'], [u'A', u'bathroom', u'scene', u'looking', u'at', u'the', u'toilet', u'and', u'the', u'sink', u'.']]]


In [3]:
def embedding_init():
    return np.random.randn(2, 30) * 0.01


def w_init():
    shape = (50, 50)
    a = np.random.normal(0.0, 1.0, shape)
    u, _, v = np.linalg.svd(a, full_matrices=False)
    q = u if u.shape == shape else v
    q = q.reshape(shape)
    return q


def u_init():
    shape = (30, 50)
    a = np.random.normal(0.0, 1.0, shape)
    u, _, v = np.linalg.svd(a, full_matrices=False)
    q = u if u.shape == shape else v
    q = q.reshape(shape)
    return q


def lr_init():
    return np.random.randn(50, ) * 0.01

In [4]:
embedding_layer = EmbeddingLayer(embedding_init)
rnn_layer = RnnLayer(w_init, u_init)
lr_layer = LogisticRegresion(lr_init)

x = T.ivector()
y = T.iscalar()

embedding_expr = embedding_layer.get_output_expr(x)
h = rnn_layer.get_output_expr(embedding_expr)
py_x = lr_layer.get_output_expr(h[-1])
y_pred = py_x > 0.5
cost = - y * T.log(py_x) - (1 - y) * T.log(1 - py_x)
updates = get_sgd_updates(cost, embedding_layer.get_parameters() + rnn_layer.get_parameters() + lr_layer.get_parameters())
train = theano.function(inputs=[x, y], outputs=[cost, y_pred], updates=updates)
val = theano.function(inputs=[x, y], outputs=[cost, y_pred])

In [5]:
c = []
acc = []
for i in xrange(200):
    for x_datum, y_datum in zip(train_x, train_y):
        a, b = train(x_datum, y_datum)
        c.append(a)
        acc.append(b == y_datum)
    if i % 10 == 0:
        print 'train', np.mean(c), np.mean(acc)
        c = []
        acc = []
        for x_datum, y_datum in zip(test_x, test_y):
            a, b = val(x_datum, y_datum)
            c.append(a)
            acc.append(b == y_datum)
        print 'val', np.mean(c), np.mean(acc)
        print '=' * 10

train 0.693215767513 0.443181818182
val 0.693036590717 0.6
train 0.692979501226 0.570786516854
val 0.692876996459 0.6
train 0.691819893263 0.64606741573
val 0.694239695534 0.7
train 0.687858924202 0.649438202247
val 0.677053153522 0.7
train 0.682752450496 0.648314606742
val 0.671618472911 0.8
train 0.662944292238 0.644943820225
val 0.586331701819 0.8
train 0.582054697441 0.683146067416
val 0.556527845575 0.7
train 0.53093086389 0.678651685393
val 0.0636801815673 1.0
train 0.0378262450133 1.0
val 0.010604075849 1.0
train 0.0124854511144 1.0
val 0.00480963121226 1.0
train 0.00708380399513 1.0
val 0.00293775584037 1.0
train 0.00483066641129 1.0
val 0.00205825514487 1.0
train 0.00361077478305 1.0
val 0.00155974550429 1.0
train 0.00285271189702 1.0
val 0.00124322888744 1.0
train 0.00233962257723 1.0
val 0.00102638603026 1.0
train 0.0019714200065 1.0
val 0.000869520665428 1.0
train 0.00169563713856 1.0
val 0.000751316610229 1.0
train 0.00148218914993 1.0
val 0.000659375743889 1.0
train 0.001